<a href="https://colab.research.google.com/github/pj-mathematician/gintama-ost-recognizer/blob/main/ostmatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install -q kurby
!pip install -q pydub
!apt install sox
!wget https://github.com/tsurumeso/vocal-remover/releases/download/v4.0.0/vocal-remover-v4.0.0.zip
!unzip -u /content/vocal-remover-v4.0.0.zip 
!cd vocal-remover && pip install -r requirements.txt
!git clone https://github.com/dpwe/audfprint
!cd audfprint && pip install -r requirements.txt

In [ ]:
def download_and_convert(ep):
  anime='gintama'
  r=!kurby download --nfrom $ep --nto $ep $anime
  print("Downloaded episode",ep,"of",anime)
  #fname=(r[1].replace(" ","")+".mp4")
  path="/content/Gintama/Gintama-S00-E00{}.mp4".format(ep)
  mname="master"+str(ep)+".wav"
  f=!ffmpeg -i $path -q:a 0 -map a $mname
  #pprint(f)
  #!rm $path
def batch_convert(eps):
  for i in range(1,eps+1):
    path="/content/Gintama/Gintama-S00-E00{}.mp4".format(i)
    mname="master{}.wav".format(i)
    f=!ffmpeg -i $path -q:a 0 -map a $mname
    print(mname,"converted")
    #print(f)
def vocal_remove(ep):
  path="/content/master{}.wav".format(ep)
  path2="/content/vocal-remover/master{}_Instruments.wav".format(ep)
  !cd vocal-remover && python inference.py --input $path --tta --postprocess
  !cp $path2 /content/drive/MyDrive/audio

In [ ]:
download_and_convert(5,"gintama")

In [ ]:
for i in range(2,7):
  vocal_remove(i)

In [ ]:
!mkdir files

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence,detect_nonsilent
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)
for j in range(1,5):
  print(j)
  fol="files{}".format(j)
  !mkdir $fol
  song = AudioSegment.from_mp3("/content/drive/MyDrive/audio/master{}_Instruments.wav".format(j))
  chunks = split_on_silence (
      song, 
      min_silence_len = 500,
      silence_thresh = -50
  )
  l = detect_nonsilent(song, min_silence_len=500,silence_thresh=-50,seek_step=1)
  iter=0
  for i, chunk in enumerate(chunks):
      silence_chunk = AudioSegment.silent(duration=500)
      s,e=l[iter]
      start=s/1000
      audio_chunk = silence_chunk + chunk + silence_chunk
      normalized_chunk = match_target_amplitude(audio_chunk, -20.0)
      print("/content/files{}/chunk{}_{}.mp3.".format(j,i,start))
      normalized_chunk.export(
          "/content/files{}/chunk{}_{}.mp3".format(j,i,start),
          bitrate = "192k",
          format = "mp3"
      )
      iter+=1

In [ ]:
!zip -r chunk.zip /content/files

In [ ]:
!cp /content/chunk.zip /content/drive/MyDrive/audio

In [ ]:
import csv
import datetime
import os
f=open("ostdata.csv","w")
writer=csv.writer(f)
writer.writerow(["Episode Number","Time Stamp","OST Name"])
for j in range(1,5):
  directory = os.fsencode("/content/files{}".format(j))
  for file in os.listdir(directory):
    filename = str(os.fsdecode(file))
    #print(filename)
    st="/content/files{}/".format(j)+filename
    #print(st)
    result =!cd audfprint && python audfprint.py match --dbase /content/drive/MyDrive/audio/ostp.pklz $st
    #for i in result:
      #print(i)
      #print()
    if result[3].startswith("Matched"):
      #print(result)
      print(st)
      a=result[3].index(" as ")
      at=result[3].index(" at ")
      name=(result[3][a+13:at-4:])
      print(name)
      un=filename.index("_")
      mp=filename.index("mp3")
      time=filename[un+1:mp-1:]
      #print(time)
      ti=((str(datetime.timedelta(seconds=int(float(time))))[2::]))
      print(ti)
      writer.writerow(["Episode {}".format(j),ti,name])
    else:
      pass
    writer.writerow(["","",""])
      #print("nope.")
f.close()

/content/files1/chunk0_0.0.mp3
Gintama Original Soundtrack 2/01 - Temeera! Hajimeruzee!
00:00
/content/files1/chunk2_28.326.mp3
Gintama Original Soundtrack 1/01 - Temee Raaaa!! Soredemo Gintama Tsuiten no Kaaaa!
00:28
/content/files1/chunk80_872.017.mp3
Gintama Original Soundtrack 1/06 - Uchuu Senkan Otousei
14:32
/content/files1/chunk124_1285.316.mp3
Gintama Original Soundtrack 1/22 - Kenkatte no wa yoo Nani ka wo Mamoru Tameni Yarumon Daro ga
21:25
/content/files1/chunk13_328.208.mp3
Gintama Original Soundtrack 1/01 - Temee Raaaa!! Soredemo Gintama Tsuiten no Kaaaa!
05:28
/content/files1/chunk112_1137.585.mp3
Gintama Original Soundtrack 1/23 - Dura Janai Katsura daaa!!
18:57
/content/files1/chunk11_212.945.mp3
Gintama Original Soundtrack 1/02 - Pray (TV version)
03:32
/content/files1/chunk29_492.37.mp3
Gintama Original Soundtrack 1/28 - Kenka wa Guu de Yarubeshi
08:12
/content/files1/chunk77_854.825.mp3
Gintama Original Soundtrack 1/06 - Uchuu Senkan Otousei
14:14
/content/files1/chu

Sun Jul 11 14:19:18 2021 Reading hash table /content/drive/MyDrive/audio/ostp.pklz
